Scraping Process

In [1]:
# import packages
from bs4 import BeautifulSoup as bs
from splinter import Browser
import requests
import pandas as pd
from time import sleep 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [6]:
# splinter setup
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

NASA Mars News

In [3]:
nasa_url_start = "https://mars.nasa.gov/news/"
browser.visit(nasa_url_start)

news_date_extract = browser.find_by_css('.list_date')[0] # remember the . for the css class
latest_news_date = bs(news_date_extract.outer_html,'lxml').select('div')[0].text.strip()  # single string to find div in bs object
latest_news_date

news_headline_extract = browser.find_by_css('.content_title')[0]
latest_news_headline = bs(news_headline_extract.outer_html,'lxml').select('div')[0].text.strip()
latest_news_headline 

news_teaser_extract = browser.find_by_css('.article_teaser_body')[0]
latest_news_teaser = bs(news_teaser_extract.outer_html,'lxml').select('div')[0].text.strip()
latest_news_teaser 

news_url_extract = browser.find_by_css('.content_title')[0]
latest_news_url_extract = bs(news_url_extract.outer_html,'lxml').select('a')[0]['href']
latest_news_url = nasa_url_start + latest_news_url_extract # string concatenation to website base url 
latest_news_url

'December  7, 2018'

"NASA InSight Lander 'Hears' Martian Winds"

'Vibrations picked up by two spacecraft instruments have provided the first sounds of Martian wind.'

'https://mars.nasa.gov/news//news/8397/nasa-insight-lander-hears-martian-winds/'

JPL Mars Space Images

In [4]:
jpl_url_start = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(jpl_url_start)
browser.click_link_by_partial_text('FULL IMAGE')
sleep(5)
browser.click_link_by_partial_text('more info')
sleep(5)
img_url = browser.find_by_css('.main_image')[0]

img_url_extract = bs(img_url.outer_html,'lxml').select('img')[0]['src'] # single string to find src in bs object
jpl_url = "https://www.jpl.nasa.gov" + img_url_extract # string concatenation to website base url 
jpl_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18848_hires.jpg'

Mars Weather

In [7]:
weather_url_start = "https://twitter.com/marswxreport?lang=en"
browser.visit(weather_url_start)

weather_date_extract = browser.find_by_css('._timestamp')[0]
tweet_weather_date = bs(weather_date_extract.outer_html,'lxml').select('span')[0].text.strip()
tweet_weather_date

# doesn't take into account other tweets, just grabs whatever the first one is
weather_report_extract = browser.find_by_css('.tweet-text')[0]
tweet_weather_report = bs(weather_report_extract.outer_html,'lxml').select('p')[0].text.strip()
tweet_weather_report

'Dec 7'

'Sol 2251 (2018-12-05), high -15C/5F, low -74C/-101F, pressure at 8.47 hPa, daylight 06:34-18:48'

In [ ]:
# may not be able to get to this...
# # trying to make for loop to run the process IF there is "Sol ", and other identifiers 

# weather_report_extract = browser.find_by_css('.stream-container')
# tweet_weather_report = bs(weather_report_extract.outer_html,'lxml')
# tweet_weather_report

# # for tweet in tweet_weather_report:

Mars Facts

In [8]:
facts_url = "https://space-facts.com/mars/"
fact_table_extract = pd.read_html(facts_url)
fact_table = fact_table_extract[0]
fact_table.columns = ['Fact', 'Value']
fact_table

Fact                          Value
0  Equatorial Diameter:                       6,792 km
1       Polar Diameter:                       6,752 km
2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
3                Moons:            2 (Phobos & Deimos)
4       Orbit Distance:       227,943,824 km (1.52 AU)
5         Orbit Period:           687 days (1.9 years)
6  Surface Temperature:                  -153 to 20 °C
7         First Record:              2nd millennium BC
8          Recorded By:           Egyptian astronomers

Mars Hemispheres

In [9]:
hemispheres_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemispheres_url)
hemi_soup_prep = browser.find_by_id('product-section')
hemi_soup = bs(hemi_soup_prep.outer_html,'lxml')

In [12]:
# extracting the names from the bs object
hemi_names = hemi_soup.find_all('h3')
hemis_name_list = []

for hemi_name in hemi_names:
    hemis_name_list.append(hemi_name.text.strip().replace(' Enhanced',''))

# extracting the relative url from the bs object and concactenating to base url
hemi_urls = hemi_soup.find_all('a')
hemi_urls_prep = []
hemi_urls_list = []

for url in hemi_urls:
    hemi_urls_prep.append(url['href'])

# from https://stackoverflow.com/a/52676102
hemi_urls_list_cleaning = pd.Series(hemi_urls_prep).drop_duplicates().tolist()

for url in hemi_urls_list_cleaning:
    hemi_urls_list.append("https://astrogeology.usgs.gov" + url)

hi_res_hemis_urls = []
# one crazy appending of a list comprehension of a bs object with selection of the a.href 
# the bs a.href grab has to be done on the same page it's being accessed on and created
for url in hemi_urls_list:
    browser.visit(url)
    sleep(1)
    hemi_url_bs = bs(browser.find_link_by_partial_text('Original').outer_html,'lxml').select('a')[0]['href']
    hi_res_hemis_urls.append(hemi_url_bs)

# zipping together the hemisphere name and img url pairings
# see https://stackoverflow.com/q/14150797
hi_res_hemis = [{'title': hemi_name, 'img_url': hi_res_hemi} for (hemi_name, hi_res_hemi) in zip(hemis_name_list, hi_res_hemis_urls)]
hi_res_hemis

[{'title': 'Cerberus Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif'}]

# Combined Outputs

In [13]:
# NASA Mars News
latest_news_date
latest_news_headline
latest_news_teaser
latest_news_url

# JPL Latest Image
jpl_url

# Twitter Mars Weather
tweet_weather_date
tweet_weather_report

# Mars Facts
fact_table

# Mars Hemispheres
hi_res_hemis

'December  7, 2018'

"NASA InSight Lander 'Hears' Martian Winds"

'Vibrations picked up by two spacecraft instruments have provided the first sounds of Martian wind.'

'https://mars.nasa.gov/news//news/8397/nasa-insight-lander-hears-martian-winds/'

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18848_hires.jpg'

'Dec 7'

'Sol 2251 (2018-12-05), high -15C/5F, low -74C/-101F, pressure at 8.47 hPa, daylight 06:34-18:48'

Fact                          Value
0  Equatorial Diameter:                       6,792 km
1       Polar Diameter:                       6,752 km
2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
3                Moons:            2 (Phobos & Deimos)
4       Orbit Distance:       227,943,824 km (1.52 AU)
5         Orbit Period:           687 days (1.9 years)
6  Surface Temperature:                  -153 to 20 °C
7         First Record:              2nd millennium BC
8          Recorded By:           Egyptian astronomers

[{'title': 'Cerberus Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif'}]